In [1]:
# SQLiteのRDBを読み込み、dfに代入
#必要なライブラリのインポート
import pandas as pd
import sqlite3
# 作成済のdatabaseに接続する。
conn = sqlite3.connect('suumo.db')

# databaseをクエリで実行した結果を読み込み、dfに代入。
query = "SELECT * FROM suumo_clean"
df = pd.read_sql(query, conn)

In [2]:
df['access1_walk/min'] = pd.to_numeric(df['access1_walk/min'], errors='coerce')
df['access2_walk/min'] = pd.to_numeric(df['access2_walk/min'], errors='coerce')
df['access3_walk/min'] = pd.to_numeric(df['access3_walk/min'], errors='coerce')

In [3]:
# addressカラムの各要素に対して末尾が1〜9であれば「丁目」を結合する
df['address'] = df['address'].apply(lambda x: x + '丁目' if x[-1] in '１２３４５６７８９' else x)
df.head()

,name,address,age/year,structure,floor/F,rent/yen,admin/yen,deposit/yen,gratuity/yen,madori,menseki/m2,access1_line,access1_station,access1_walk/min,access2_line,access2_station,access2_walk/min,access3_line,access3_station,access3_walk/min
0,メゾンクレール,東京都北区田端３丁目,38,7F,5,125000,0,125000,125000,2DK,45.50,ＪＲ山手線,駒込駅,8.0,ＪＲ京浜東北線,田端駅,12.0,東京メトロ千代田線,西日暮里駅,16.0
1,ＪＲ山手線 鶯谷駅 10階建 築3年,東京都荒川区東日暮里４丁目,3,10F,3,190000,10000,190000,190000,2LDK,50.08,ＪＲ山手線,鶯谷駅,6.0,ＪＲ京浜東北線,日暮里駅,11.0,ＪＲ山手線,上野駅,22.0
2,イルメーゼ,東京都北区田端４丁目,35,5F,3,120000,5000,120000,120000,2DK,42.44,ＪＲ山手線,田端駅,8.0,ＪＲ山手線,西日暮里駅,19.0,ＪＲ山手線,駒込駅,8.0
3,田端３丁目戸建,東京都北区田端３丁目,43,2F,-,200000,0,200000,0,3DK,53.21,ＪＲ山手線,田端駅,10.0,東京メトロ千代田線,西日暮里駅,13.0,東京メトロ南北線,駒込駅,12.0
4,ＪＲ山手線 田端駅 3階建 築33年,東京都北区田端５丁目,33,3F,2,132000,3000,132000,0,3SDK,58.94,ＪＲ山手線,田端駅,8.0,ＪＲ山手線,駒込駅,11.0,ＪＲ山手線,西日暮里駅,16.0


In [4]:
from geopy.geocoders import Nominatim
import pandas as pd
import tqdm

geolocator = Nominatim(user_agent="suumo", timeout=10)

# 住所データ
addresses = df['address']  # ここに住所リスト

# 緯度経度に変換
locations = []
for address in tqdm.tqdm(addresses):
    try:
        location = geolocator.geocode(address)
        if location:
            locations.append((location.latitude, location.longitude))
        else:
            locations.append((None, None))
    except Exception as e:
        locations.append((None, None))

df_ = pd.DataFrame(locations, columns=['lat', 'lon'])


100%|██████████| 3008/3008 [33:24<00:00,  1.50it/s]  


In [5]:
df = pd.concat([df,df_], axis=1)
df

,name,address,age/year,structure,floor/F,rent/yen,admin/yen,deposit/yen,gratuity/yen,madori,...,access1_station,access1_walk/min,access2_line,access2_station,access2_walk/min,access3_line,access3_station,access3_walk/min,lat,lon
0,メゾンクレール,東京都北区田端３丁目,38,7F,5,125000,0,125000,125000,2DK,...,駒込駅,8.0,ＪＲ京浜東北線,田端駅,12.0,東京メトロ千代田線,西日暮里駅,16.0,35.740101,139.760583
1,ＪＲ山手線 鶯谷駅 10階建 築3年,東京都荒川区東日暮里４丁目,3,10F,3,190000,10000,190000,190000,2LDK,...,鶯谷駅,6.0,ＪＲ京浜東北線,日暮里駅,11.0,ＪＲ山手線,上野駅,22.0,35.726600,139.780900
2,イルメーゼ,東京都北区田端４丁目,35,5F,3,120000,5000,120000,120000,2DK,...,田端駅,8.0,ＪＲ山手線,西日暮里駅,19.0,ＪＲ山手線,駒込駅,8.0,35.745089,139.744639
3,田端３丁目戸建,東京都北区田端３丁目,43,2F,-,200000,0,200000,0,3DK,...,田端駅,10.0,東京メトロ千代田線,西日暮里駅,13.0,東京メトロ南北線,駒込駅,12.0,35.740101,139.760583
4,ＪＲ山手線 田端駅 3階建 築33年,東京都北区田端５丁目,33,3F,2,132000,3000,132000,0,3SDK,...,田端駅,8.0,ＪＲ山手線,駒込駅,11.0,ＪＲ山手線,西日暮里駅,16.0,35.740101,139.760583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,豊島区雑司が谷２丁目新築計画,東京都豊島区雑司が谷２丁目,0,2F,2,189000,3000,189000,189000,2LDK,...,雑司が谷駅,5.0,東京メトロ有楽町線,東池袋駅,14.0,ＪＲ山手線,池袋駅,20.0,35.719109,139.716289
3004,ラ・カーサ広尾,東京都渋谷区東４丁目,0,7F,3,207000,18000,207000,0,2LDK,...,恵比寿駅,15.0,東京メトロ千代田線,表参道駅,18.0,東京メトロ日比谷線,広尾駅,19.0,35.655588,139.712620
3005,ラ・カーサ広尾,東京都渋谷区東４丁目,0,7F,3,217000,18000,217000,0,2LDK,...,恵比寿駅,15.0,東京メトロ千代田線,表参道駅,18.0,東京メトロ日比谷線,広尾駅,19.0,35.655588,139.712620
3006,Ｈａｌｅ Ｎａｎａｌａ,東京都豊島区南池袋３丁目,0,3F,1,203000,6500,203000,406000,2LDK,...,池袋駅,4.0,東京メトロ有楽町線,東池袋駅,8.0,東京メトロ副都心線,雑司が谷駅,10.0,35.724315,139.714282


In [25]:
df[['address', 'lat', 'lon']].query('lat != lat')

,address,lat,lon
52,神奈川県横浜市磯子区上町,NaN,NaN
410,東京都目黒区三田２丁目,NaN,NaN
697,東京都目黒区三田２丁目,NaN,NaN
698,東京都目黒区三田２丁目,NaN,NaN
977,東京都目黒区三田２丁目,NaN,NaN
1060,東京都台東区三筋１丁目,NaN,NaN
1100,東京都目黒区三田１丁目,NaN,NaN
1207,東京都目黒区三田２丁目,NaN,NaN
1543,東京都台東区三ノ輪１丁目,NaN,NaN
1620,東京都台東区三ノ輪１丁目,NaN,NaN


In [23]:
geolocator = Nominatim(user_agent="suumo", timeout=10)

# 住所データ
addresses = ['東京都港区三田丁目']  # ここに住所リスト

# 緯度経度に変換
locations = []
for address in tqdm.tqdm(addresses):
    try:
        location = geolocator.geocode(address)
        if location:
            locations.append((location.latitude, location.longitude))
        else:
            locations.append((None, None))
    except Exception as e:
        locations.append((None, None))
df_ = pd.DataFrame(locations, columns=['lat', 'lon'])
df_

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


,lat,lon
0,None,None


In [8]:
#必要なライブラリのインポート
import sqlite3
# databaseに接続する。指定したdatabaseが存在しない場合は、新規作成される。
conn = sqlite3.connect('suumo_loc.db')
# 物件情報が入ったdfをsqlに書き出す。
df.to_sql('suumo_loc', conn, if_exists='replace', index=False)

3008